# Define Audit Process

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [96]:
%connect name=Transcend

Success: 'Transcend' connection established


In [99]:
call adlste_coa.sp_meta_objects_refresh('none');

Success: 0 rows affected

In [98]:
Select * from adlste_coa.coa_meta_objects
where ObjFunction  = 'audit' 
order by ObjType, ProcessName,  ObjType desc;

DatabaseName,TableName,TableKind,ObjType,ObjFunction,ProcessName
ADLSTE_COA,coa_Audit_Issues,V,coa,Audit,Issues
ADLSTE_COA,coa_audit_run,V,coa,audit,run
ADLSTE_COA,coa_audit_runlog,V,coa,audit,runlog
ADLSTE_COA,coa_Audit_Status,V,coa,Audit,Status
ADLSTE_COA,coa_audit_summary,V,coa,audit,summary
ADLSTE_COA,coat_Audit_Issues,T,coat,Audit,Issues
ADLSTE_COA,coat_audit_runlog,T,coat,audit,runlog
ADLSTE_COA,coat_Audit_Status,T,coat,Audit,Status
ADLSTE_COA,sp_audit_Account_Name,P,sp,audit,Account_Name
ADLSTE_COA,sp_audit_log,P,sp,audit,log


<br/><br/><br/>
## Audit_Run (sub-stored_procedure)
### for tracking audit columns per row while minimizing storage
- database:    adlste_coa
- table:       coat_audit_run
- view:        coa_audit_run
- stored proc: sp_audit_run

In [ ]:
drop table adlste_coa.coat_audit_runlog

In [ ]:
Create multiset table adlste_coa.coat_audit_runlog
(
 Run_ID       INTEGER 
,ParentRun_ID INTEGER
,Site_ID      VARCHAR(128)
,TableName    VARCHAR(128)
,StartDate    DATE
,EndDate      DATE
,CallingSP    VARCHAR(128)
,SQLVersion   VARCHAR(128)
,Run_Type     VARCHAR(128)
,Run_Status   VARCHAR(128)
,Run_User     VARCHAR(128) Default USER
,Run_TS       Timestamp(0) Default Current_Timestamp(0)
) 
primary index (Site_ID, TableName);

In [ ]:
replace view adlste_coa.coa_audit_runlog 
as locking row for access
Select * from adlste_coa.coat_audit_runlog

In [ ]:
replace PROCEDURE adlste_coa.sp_audit_log (inout runid integer
                                              ,in siteid varchar(128)
                                              ,in tblname varchar(128)
                                              ,in callingsp varchar(128)
                                              ,in sqlversion varchar(128)
                                              ,in startdate date
                                              ,in enddate date
                                              ,in runtype varchar(128)
                                              ,in runstatus varchar(128)
                                              )
SQL SECURITY CREATOR
BEGIN
DECLARE runidout INTEGER;

    SELECT coalesce(Max(Run_ID)+1,1) as Run_ID   INTO runidout 
    FROM adlste_coa.coat_audit_runlog 
    ;
    
    Insert into adlste_coa.coat_audit_runlog 
    values
    ( :runidout
     ,:runid --parentrunid
     ,:siteid
     ,:tblname
     ,:startdate
     ,:enddate
     ,:callingsp
     ,:sqlversion
     ,:runtype
     ,:runstatus
     ,USER 
     ,Current_Timestamp(0)
    );
    
    SET runid = runidout ;

END;

In [ ]:
-- delete from adlste_coa.coat_audit_runlog -- don't do this

### Example of some calls - designed to be embedded in other SPs

In [ ]:
-- start SP()
CALL adlste_coa.sp_audit_log(0
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'START' );  -- Run Status

In [ ]:
-- do stuff, but need a checkpoint 
-- (number above becomes last parameter below)
call adlste_coa.sp_audit_log(4
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'CHECKPOINT' );  -- Run Status

In [ ]:
-- end of SP()
-- (number above becomes last parameter below)
call adlste_coa.sp_audit_log(4
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'END'); -- Run Status

In [ ]:
Select * from adlste_coa.coat_audit_runlog

In [ ]:
replace view adlste_coa.coa_audit_summary   as locking row for access
Select 
 case when ParentRun_ID=0 then Run_ID else ParentRun_ID end as Run_ID
,Site_ID
,min(cast(Run_TS as DATE)) as LogDate
,TableName
,CallingSP
,SQLVersion
,Run_Type
,count(*) as Steps
,sum(case when Run_Status='ERROR' then 1 else 0 end) as Errors
,sum(case when Run_Status='WARNING' then 1 else 0 end) as Warnings
,min(case when Run_Status='START' then Run_TS else NULL end) as Start_TS
,max(case when Run_Status='END' then Run_TS else NULL end) as End_TS
from adlste_coa.coa_audit_runlog
Group by 1,2,4,5,6,7

In [ ]:
select * from adlste_coa.coa_audit_summary order by 1